# Task 2

### Removing pdf_url from given page_url in the excel sheet

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df=pd.read_csv('C:/Users/aamir/urls.csv')
df.head()
df=df[['URLS']]
df['pdf_url']=''  #Creating a column to store pdf url
df['name']=''     #Creating a column to store name of pdf

  
def get_pdf_url_and_pdf_name(url):
    if '.pdf' in url:                         #If url has .pdf then page and pdf url will be same
        split_url = url.split('/')
        filename = split_url[-1][:-4]+'.pdf'  #Splitting the url at every ' ' and extracting the name of the book
        return url, filename
    else:                                     #if .pdf is not in url then looking for it in the HTMl code
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.findAll('a',{'class':'format-summary download-pill'}) #looking for anchor tags with this particular class
        for link in links:                                                 # to get the download link
            if ('.pdf' in link.get('href', [])):
                href = 'https://archive.org'+link.get('href')
                split_url = href.split('/')
                filename = split_url[-1][:-4]+'.pdf'
                return href, filename

for i in range(len(df)): #Note for 46th url given in the excel sheet the class attribute was different                                                                   
    if i==46:            # Therefore I have skipped the 46th URl over here. I performed OCR separately for this book and appended it to the JSON
        continue
    else:
        df.loc[i,'pdf_url'],df.loc[i,'name']=download_pdfs(df.loc[i,'URLS'])

### Downloading pdf's from the given urls

In [ ]:
def download_pdfs(url):       #Same as above but this function is used to download the pdf
    if '.pdf' in url:
        split_url = url.split('/')
        filename = split_url[-1][:-4]+'.pdf'
        response = requests.get(url)
        pdf = open('C:/Users/aamir/Marathi_pdfs/'+filename, 'wb')
        pdf.write(response.content)
        pdf.close()
        #return url, filename
    else:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.findAll('a',{'class':'format-summary download-pill'}) 
        for link in links:
            if ('.pdf' in link.get('href', [])):
                href = 'https://archive.org'+link.get('href')
                split_url = href.split('/')
                filename = split_url[-1][:-4]+'.pdf'
                response = requests.get(href)
                pdf = open('C:/Users/aamir/Marathi_pdfs/'+filename, 'wb')
                pdf.write(response.content)
                pdf.close()
                #return href, filename
                
df['URLS'].apply(download_pdfs)

### Converting PDF's to IMG and applying OCR using Tesseract. Storing the results as a JSON file

In [ ]:
import json
from PIL import Image
from pytesseract import image_to_string  #Used for OCR 
import sys
from pdf2image import convert_from_path
import os
import requests


def convert_pdf_to_img(pdf_file):   #First step is to convert pdf to image format
   
    return convert_from_path(pdf_file,poppler_path='C:/Users/aamir/Downloads/poppler-0.68.0/bin')


def convert_image_to_text(file): #Using to tesseract to apply OCR to convert image to text.
    
   
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
    text = image_to_string(file, lang='mar') #language is marathi
    return text


def get_text_from_any_pdf(path,page_url,pdf_url): #This function takes in the pdf path, page_url and pdf_url
    
    images = convert_pdf_to_img(path)
    final_text = ""
    book_json={}
    result={}
    for pg, img in enumerate(images):
        
        final_text = convert_image_to_text(img)   #For each page we get the extracted text
        #print("Page n°{}".format(pg))
        #print(convert_image_to_text(img))
        book_json[pg]=final_text                #Created a dictionary with pg no. as key and extracted text as value
    result['page_url'] = page_url               #The resultant dictionary contains the required output in the specified format
    result['pdf_url'] = pdf_url
    result['paragraph'] = book_json             #The paragraph contains entire content of pdf with each page and its text
    return result

json_result=[]
for x in range(len(df)):
    print(x)
    if i==46:                 #Note for 46th url given in the excel sheet the class attribute was different
        continue              # Therefore I have skipped the 46th URl over here. I performed OCR separately for this book and appended it to the JSON
    else:
        filename = df['name'][x]
        json_result.append(get_text_from_any_pdf('C:/Users/aamir/Marathi_pdfs/'+filename,df['URLS'][x],df['pdf_url'][x]))


with open('pdf_extract.json', 'w', encoding='utf8') as file: #Saving as json file
    file.write(str(json_result))